In [3]:
!pip install --upgrade numpy
!pip install --upgrade pandas
#!pip install pyspark==2.4.5
!pip install -U scikit-learn
#!pip install gensim
!pip install -U keras
# we want tensorflow 2.3
!pip install --upgrade tensorflow


Requirement already up-to-date: numpy in c:\users\adamdavis\documents\projects\data-science-advanced-capstone\.venv\lib\site-packages (1.19.2)
Requirement already up-to-date: pandas in c:\users\adamdavis\documents\projects\data-science-advanced-capstone\.venv\lib\site-packages (1.1.2)
Requirement already up-to-date: scikit-learn in c:\users\adamdavis\documents\projects\data-science-advanced-capstone\.venv\lib\site-packages (0.23.2)


In [1]:
import tensorflow as tf
print("Tensorflow version: ", tf.__version__)
if not tf.__version__ == '2.3.0':
    raise ValueError('please upgrade to TensorFlow 2.3, or restart your Kernel (Kernel->Restart & Clear Output)')
from tensorflow import keras


Tensorflow version:  2.3.0


In [2]:
#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
from pprint import pprint
from time import time
import logging
import numpy as np
import pandas as pd
import string
import re
from datetime import datetime
#from packaging import version
from ibm_botocore.client import Config
import ibm_boto3
pd.__version__
#from sklearn.model_selection import train_test_split

'1.1.2'

In [8]:
pip install ibm-cos-sdk

Processing c:\users\adamdavis\appdata\local\pip\cache\wheels\56\14\78\c2fc4fe36e1e7198e4bde8707cf59b838a7be6859c631142f6\ibm_cos_sdk-2.7.0-py2.py3-none-any.whl
Processing c:\users\adamdavis\appdata\local\pip\cache\wheels\bf\bf\71\d4b460c36adadb88a24dfd26411c8b4dfa19459d15988c648d\ibm_cos_sdk_s3transfer-2.7.0-py2.py3-none-any.whl
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
Processing c:\users\adamdavis\appdata\local\pip\cache\wheels\54\a5\95\ac2068077b0f41719c2a69a03fdfbdd76ad02cdbb0984ad1a7\ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl
  Using cached docutils-0.15.2-py3-none-any.whl (547 kB)
  Using cached requests-2.24.0-py2.py3-none-any.whl (61 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached urllib3-1.25.10-py2.py3-none-any.whl (127 kB)
  Using cached certifi-2020.6.20-py2.py3-none-any.whl (156 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
#Get our data from IBM Cloud

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_news = {
    'IAM_SERVICE_ID': 'iam-ServiceId-32e8ee67-397c-4ff1-b69b-543172331f43',
    'IBM_API_KEY_ID': 'Rx4FR4JSAueCnnIsoevsgYgOsuh8LCXtbkFpFpC0EmVU',
    #'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'ENDPOINT':'https://s3-api.us-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'advanceddatasciencecapstone-donotdelete-pr-tqabpnbxebk8rm',
    'FILE': 'dfTrueFalseNews.pkl'
}

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print(' File Uploaded')
        
dfNews = download_file_cos(credentials_news, "dfTrueFalseNews.pkl", "dfTrueFalseNews.pkl")
#dfNews = download_file_cos(credentials_news, "conf.yml", "conf.yml")
#dfNews = download_file_cos(credentials_news, "pipeline.py", "pipeline.py")
#dfNews = download_file_cos(credentials_news, "text_preprocessing.py", "text_preprocessing.py")
#dfNews = download_file_cos(credentials_news, "RNN_model.py", "RNN_model.py")


File Downloaded


In [3]:
dfNewsTemp = pd.read_pickle('dfTrueFalseNews.pkl')
#dfTrueFalseNews_tokenized = pd.read_pickle('dfTrueFalseNews_tokenized.pkl')
#dfNews['truthvalue'] = pd.Categorical(dfNews['truthvalue'])
type(dfNewsTemp)

x = dfNewsTemp['text'].values
y = dfNewsTemp['truthvalue'].values
print (dfNewsTemp.shape, dfNewsTemp.columns, '\n', dfNewsTemp.dtypes, type(x), type(y))

(1126, 3) Index(['text', 'source', 'truthvalue'], dtype='object') 
 text          object
source        object
truthvalue    object
dtype: object <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [8]:
# Get the pretrained GloVe embedding.
#!curl http://nlp.stanford.edu/data/glove.6B.zip --output glove.6B.zip -L
download_file_cos(credentials_news, "glove.6B.zip", "glove.6B.zip")
#!ls -al
!unzip glove.6B.zip glove.6B.300d.txt
#!ls -al
#upload_file_cos(credentials_news, "glove.6B.zip", "glove.6B.zip")
!rm glove.6B.zip
!ls -al

KeyboardInterrupt: 

In [4]:
# Package for data wrangling
import pandas as pd 
# Package for array math
import numpy as np 
# Package for system path traversal
import os
# Package for working with dates
from datetime import date
# K fold analysis package
from sklearn.model_selection import KFold
# Import the main analysis pipeline
from pipeline import Pipeline
# Tensor creation class
from text_preprocessing import TextToTensor


In [5]:
conf = {'k_fold': True,
         'save_results': True,
         'batch_size': 256,
         'epochs': 7,
         'max_len': 20}

In [33]:
import importlib
import pipeline
importlib.reload(pipeline)

<module 'pipeline' from 'c:\\Users\\AdamDavis\\Documents\\Projects\\Data-Science-Advanced-Capstone\\pipeline.py'>

In [6]:
# Modified, based on https://github.com/Eligijus112/twitter-genuine-tweets/blob/master/master.py
# We're not going to use any stop words. The pretrained embedding has a lot of words.
stop_words = []
#try:
#    stop_words = pd.read_csv('data/stop_words.txt', sep='\n', header=None)[0].tolist()
#except Exception as e:
    # This exception indicates that the file is missing or is in a bad format
#    print('Bad stop_words.txt file: {e}')

# Reading the data
#train = pd.read_csv('data/train.csv')[['text', 'target']]
#test = pd.read_csv('data/test.csv')

#from sklearn import model_selection
#Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dfNewsTemp['text'],dfNewsTemp['truthvalue'],test_size=0.1)

# Creating the input for the pipeline
X_train = dfNewsTemp['text'].tolist()
Y_train = dfNewsTemp['truthvalue'].tolist()

X_test = dfNewsTemp['text'].tolist()
fit_history = []
if conf.get('k_fold'):
    kfold = KFold(n_splits=4)
    acc = []
    f1 = []
    history = []
    for train_index, test_index in kfold.split(X_train):
        # Fitting the model and forecasting with a subset of data
        k_results = Pipeline(
            X_train=np.array(X_train)[train_index],
            Y_train=np.array(Y_train)[train_index], 
            embed_path='glove.6B.300d.txt',
            embed_dim=300,
            X_test=np.array(X_train)[test_index],
            Y_test=np.array(Y_train)[test_index],
            max_len=conf.get('max_len'),
            epochs=conf.get('epochs'),
            batch_size=conf.get('batch_size')
        )
        # Saving the accuracy
        acc += [k_results.acc]
        f1 += [k_results.f1]
        fit_history.append(k_results.fit_history)
        history.append(k_results)
        print(f'The accuracy score is: {acc[-1]}') 
        print(f'The f1 score is: {f1[-1]}') 
    print(f'Total mean accuracy is: {np.mean(acc)}')
    print(f'Total mean f1 score is: {np.mean(f1)}')

# Running the pipeline with all the data
results = Pipeline(
    X_train=X_train,
    Y_train=Y_train, 
    embed_path='glove.6B.300d.txt',
    embed_dim=300,
    #stop_words=stop_words,
    X_test=X_test,
    #Y_test=Y_test,
    max_len= None ,#   conf.get('max_len'),
    epochs=conf.get('epochs'),
    batch_size=conf.get('batch_size')
)

# Some sanity checks
#good = ["Fire in Vilnius! Where is the fire brigade??? #emergency"]
#bad = ["Sushi or pizza? Life is hard :(("]

TextToTensor_instance = TextToTensor(
tokenizer=results.tokenizer,
max_len=conf.get('max_len')
)

# Converting to tensors
good_nn = TextToTensor_instance.string_to_tensor(good)
bad_nn = TextToTensor_instance.string_to_tensor(bad)

# Forecasting
p_good = results.model.predict(good_nn)[0][0]
p_bad = results.model.predict(bad_nn)[0][0]

print(f'Sentence: {good_nn} Score: {p_good}')
print(f'Sentence: {bad_nn} Score: {p_bad}')

# Saving the predictions
test['prob_is_genuine'] = results.yhat
test['target'] = [1 if x > 0.5 else 0 for x in results.yhat]
 
# Saving the predictions to a csv file
if conf.get('save_results'):
    if not os.path.isdir('output'):
        os.mkdir('output')    
    test[['id', 'target']].to_csv(f'output/submission_{date.today()}.csv', index=False)

results.save("revisedDeepLearningPretrainedEmbed.model")
upload_file_cos(credentials_news, "revisedDeepLearningPretrainedEmbed.model", "revisedDeepLearningPretrainedEmbed.model")

import pickle
filename = "revisedDeepLearningPretrainedEmbed.history"
with open(filename, 'wb') as f:
    pickle.dump(history, f)

Epoch 1/7
4/4 [==============================] - ETA: 0s - loss: 0.6605WARNING:tensorflow:Model was constructed with shape (None, 20) for input Tensor("input_1:0", shape=(None, 20), dtype=float32), but it was called on an input with incompatible shape (None, 1).


UnimplementedError:  Cast string to float is not supported
	 [[node functional_1/Cast (defined at c:\Users\AdamDavis\Documents\Projects\Data-Science-Advanced-Capstone\pipeline.py:57) ]] [Op:__inference_test_function_6748]

Function call stack:
test_function


In [8]:


import text_preprocessing

Using TensorFlow backend.
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/Python36/lib/python3.6/site-packag

In [ ]:
pd.__version__

  Using cached scipy-1.4.1-cp38-cp38-win_amd64.whl (31.0 MB)
  Using cached Werkzeug-1.0.1-py2.py3-none-any.whl (298 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)
Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\AdamDavis\\Documents\\Projects\\Data-Science-Advanced-Capstone\\.venv\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

